In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt

In [49]:
TrafikDataDF=pd.read_excel('İstanbulTraficData.xlsx',sheet_name="SimpleData")

In [50]:
Days=pd.to_datetime("today").date()-TrafikDataDF.TrafficIndexDate.dt.date.max()

In [51]:
Days.days

6

In [53]:
#IBB veri setinden son 910 günün verisini 30 Dakika aralıklı olarak çekiyoruz
response = requests.get("https://api.ibb.gov.tr/tkmservices/api/TrafficData/v1/TrafficIndexHistory/"+str(Days.days)+"/30M")
getData=pd.DataFrame(response.json())
TrafikDataDF=pd.concat(TrafikDataDF,getData)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [12]:
a=TrafikDataDF.groupby('TrafficIndexDate').TrafficIndexDate.max()

datetime.timedelta(days=-910)

datetime.date(2022, 11, 6)

In [10]:
pd.set_option("display.max_rows", 5)

In [ ]:
TrafikDataDF1=TrafikDataDF

In [6]:
TrafikDataDF['TrafficIndexDate']=TrafikDataDF['TrafficIndexDate'].apply(pd.to_datetime)

TrafikDataDF['Date'] = TrafikDataDF.TrafficIndexDate.dt.date
TrafikDataDF['Time'] = TrafikDataDF.TrafficIndexDate.dt.time
TrafikDataDF["Day"]=TrafikDataDF.TrafficIndexDate.dt.day
TrafikDataDF["Month"]=TrafikDataDF.TrafficIndexDate.dt.month
TrafikDataDF["Year"]=TrafikDataDF.TrafficIndexDate.dt.year
TrafikDataDF["Week"]=TrafikDataDF.TrafficIndexDate.dt.week


In [ ]:

TrafikDataDF1['daysofweek'] = TrafikDataDF1['TrafficIndexDate'].dt.dayofweek
TrafikDataDF1["day_name"]=TrafikDataDF1.TrafficIndexDate.dt.day_name()

In [ ]:
TrafikDataDF['is_weekend']=TrafikDataDF1['is_weekend']

In [ ]:
TrafikDataDF1['is_weekend'] = np.where(TrafikDataDF1['day_name'].isin(['Sunday', 'Saturday']), 1,0)

Tatil Günleri Hesaplama

In [ ]:
TatilGunleri=pd.read_csv("./Turkish calendar.csv",sep=';')

In [ ]:
Date=TrafikDataDF.drop_duplicates(subset=['tarih'])


In [ ]:
TatilGunleri.columns

In [ ]:
TrafikDataDF.to_excel('İstanbulTraficData.xlsx')

In [ ]:
TatilGunleri['CALENDAR_DATE']=TatilGunleri['CALENDAR_DATE'].apply(pd.to_datetime)

In [ ]:
TatilGunleri['Ay']=TatilGunleri.CALENDAR_DATE.dt.month
TatilGunleri['Yıl']=TatilGunleri.CALENDAR_DATE.dt.year
TatilGunleri['Gün']=TatilGunleri.CALENDAR_DATE.dt.day
TatilGunleri['Hafta']=TatilGunleri.CALENDAR_DATE.dt.week
TatilGunleri['Tarih'] = TatilGunleri.CALENDAR_DATE.dt.date

In [ ]:
TatilGunleri.to_excel('TatilData.xlsx')